### Parsing and Preprocessing Index Pages

- filter tiny and huge text elements (i.e. deviating from average character/word width and height
- extract page lines that are part of the main text body containing index entries
- insert and clean up repetition symbols in index entries
    - determine length of repetition symbol
    - identify and replace mis-recognized repetition symbols


In [3]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [17]:
from collections import defaultdict

from republic.model.republic_hocr_model import make_hocr_page
import republic.parser.republic_column_parser as column_parser
from elasticsearch import Elasticsearch
import republic.parser.republic_page_parser as page_parser
import republic.parser.republic_paragraph_parser as paragraph_parser
import republic.parser.republic_file_parser as file_parser
from republic.config.republic_config import base_config, set_config_year


es = Elasticsearch()

year = 1725
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"


def get_pages_info(config):
    scan_files = file_parser.get_files(config["data_dir"])
    print("Number of scan files:", len(scan_files))
    return file_parser.gather_page_columns(scan_files)

year_config = set_config_year(base_config, year, data_dir)
pages_info = get_pages_info(year_config)



Number of scan files: 2161


In [26]:
#from republic_index_page_parser import index_lemmata
from collections import defaultdict
import republic.parser.republic_index_page_parser as index_parser
import republic.elastic.republic_elasticsearch as rep_es

avg_left = 0
lemma_index = defaultdict(list)
curr_lemma = None
    

for page_id in pages_info:
    page_doc = rep_es.retrieve_page_doc(es, page_id, year_config)
    print("\n\n", page_id)
    if page_doc["page_type"] != "index_page":
        print("skipping non-index page")
        continue
    page_doc["num_page_ref_lines"] = index_parser.count_page_ref_lines(page_doc)
    for column_hocr in page_doc["columns"]:
        lines = index_parser.get_index_entry_lines(column_hocr)
        curr_lemma = index_parser.index_lemmata(lines, lemma_index, curr_lemma)
        print("returned lemma:", curr_lemma)






 year-1725-scan-1-odd
skipping non-index page


 year-1725-scan-4-odd
skipping non-index page


 year-1725-scan-5-odd
possibly misrecognised repeat symbol: 1 149 a
possibly misrecognised repeat symbol: 9 137 mm
possibly misrecognised repeat symbol: 11 145 a
possibly misrecognised repeat symbol: 17 149 ms
DEVIATING LINE: 265 [146, 98, 169, 265, 272, 144, 98, 140] Tien ‘toegeleght. 455.
avg_repeat_symbol_length: 145
DEVIATING LINE: 272 [146, 98, 169, 265, 272, 144, 98, 140, 144] jn Vetalige ‘van achterflallen , Te
avg_repeat_symbol_length: 145
DEVIATING LINE: 239 [98, 147, 150, 99, 239] tot Amfterdam. 237.
avg_repeat_symbol_length: 145
	PAGE_REFS: [256] 	CURR LEMMA: None
0 146 22 continue_stop 	        vincien ver/ocht ‘confentenYe            256.
1 98 -29 start 	      ——  de  Haas  ‘om  ‘a(fijtertie  ‘afgeweefen.
	PAGE_REFS: [10] 	CURR LEMMA: None
2 169 46 continue_stop 	         10,
	PAGE_REFS: [455] 	CURR LEMMA: None
3 100 -25 start_stop 	      —— Tien  ‘toegeleght.   455.
4 107 -20

9 33 -27 start 	   —— aangefchreven de Jenny Galley te Tas
	PAGE_REFS: [591] 	CURR LEMMA: Admiraliteyt tot Amfterdam
10 78 17 continue_stop 	     ten loen.   591.
11 36 -20 start 	   —— propofitie vm twee Commien Generaal
	PAGE_REFS: [637] 	CURR LEMMA: Admiraliteyt tot Amfterdam
12 81 20 continue_stop 	     aan te tellen, geaccordeert.  637.
13 34 -22 start 	   ——— Buys en vander Meulen aangeftelt tot
	PAGE_REFS: [639] 	CURR LEMMA: Admiraliteyt tot Amfterdam
14 86 24 continue_stop 	     Commifen Generaal.  639.
15 42 -21 start 	   —— commijie  wegens  Hollandt voor den
	PAGE_REFS: [716] 	CURR LEMMA: Admiraliteyt tot Amfterdam
16 84 20 continue_stop 	     Heer vander Waeyen.   716.
17 37 -28 start 	   ——— Lijfte  van  inkoomende  Rundvee  en
	PAGE_REFS: [786] 	CURR LEMMA: Admiraliteyt tot Amfterdam
18 87 16 continue_stop 	     Varckens.  786,
	PAGE_REFS: [896] 	CURR LEMMA: Admiraliteyt tot Amfterdam
19 88 16 continue_stop 	     vangh van fijn bedieninge gedaan.  896.
20 40 -33 start 	  

setting lemma: de
0 107 -27 start 	      de Baudous om de              van ’s Lands
1 156 19 continue 	         Vloot, de [elve voor eerft omvergeven te la-
	PAGE_REFS: [266] 	CURR LEMMA: de
2 155 23 continue_stop 	         ten.   266.
HAS LEMMA: Beaufort aangefteldt tot Secretaris , Schoolmee-
LEMMA: Beaufort aangefteldt tot Secretaris 
LEMMA: Beaufort aangefteldt tot Secretaris 
setting lemma: Beaufort aangefteldt tot Secretaris
3 107 -27 start 	      Beaufort aangefteldt tot Secretaris, Schoolmee-
4 147 16 continue 	        fer  en  Kofter van  de Heerlijkheden  van
	PAGE_REFS: [166] 	CURR LEMMA: Beaufort aangefteldt tot Secretaris
5 150 19 continue_stop 	        gulden ter          fourneerende.   166.
HAS LEMMA: Becker, creditif van den Furft van Ooftvries-
LEMMA: Becker
LEMMA: Becker
setting lemma: Becker
6 104 -25 start 	      Becker, creditif van den Furft van Ooftvries-
	PAGE_REFS: [31] 	CURR LEMMA: Becker
7 150 27 continue_stop 	        landt.   31.
	PAGE_REFS: [96] 	CURR LEM

11 119 -32 start 	       —— Regenten  van  de Hage verfoeckende
12 161 1 continue 	         dat het Oortjesgeldt dat in de Herbergen in
13 163 -2 start 	         de           werdt gegeven  aan de Diato-
14 162 1 continue 	         nye Armen magh komen,  en dat den Pre-
15 159 -3 start 	         dikant fem        hebben  in  het  verkie-
16 243 76 continue 	             van  cen  Armmeefter 3: te examineeren.
	PAGE_REFS: [688] 	CURR LEMMA: Borgers
17 163 2 continue_stop 	         688.
18 129 -32 start 	       —— DrûfJard gelaft ordre te       tot bet
19 162 1 continue 	         opvangen  van  [Heydens  en  Vagabonden.
	PAGE_REFS: [692] 	CURR LEMMA: Borgers
20 163 4 continue_stop 	         692.
21 113 -36 start 	       —— Regenten van de Hage wegens [teke-
22 159 10 continue 	         ve Capitalen, de Raadt van Staate te ad-
	PAGE_REFS: [717] 	CURR LEMMA: Borgers
23 163 10 continue_stop 	         vijeeren.  717.
24 138 -14 start 	       —— Schouten ende Regenten ,  mitsgaders
25 157 10 

HAS LEMMA: Bronckhorft, Regquefte antidotaal ter Griffie
LEMMA: Bronckhorft
LEMMA: Bronckhorft
setting lemma: Bronckhorft
51 55 -28 start 	    Bronckhorft,  Regquefte  antidotaal ter Griffie
	PAGE_REFS: [297] 	CURR LEMMA: Bronckhorft
52 105 23 continue_stop 	      te [eponeeren.   297:
HAS LEMMA: de Brofe, Ontfanger Hogendorp gelaft quitan-
LEMMA: de Brofe
LEMMA: de Brofe
setting lemma: de Brofe
53 53 -22 start 	    de Brofe, Ontfanger Hogendorp gelaft quitan-
54 98 19 continue 	      tie te            3.                      A
55 52 -21 start 	    — wegens de forme van de quitantie„ den:
	PAGE_REFS: [170] 	CURR LEMMA: de Brofe
56 99 27 continue_stop 	      Ontfanger Generaal 1e berichten.   170.
57 50 -18 start 	   mn bericht ,  de  retroafla  ma  tt  fien.
	PAGE_REFS: [182] 	CURR LEMMA: de Brofe
58 101 31 continue_stop 	      182:
	PAGE_REFS: [187] 	CURR LEMMA: de Brofe
59 48 -17 start_stop 	   rapport te examineeren.   187.
	PAGE_REFS: [227] 	CURR LEMMA: de Brofe
60 45 -23 start_sto

...

4 9 -58 start 	 oo   —     voor Verhoef om de Monteeringe voor
5 100 39 continue 	      bet           van Reghteren  na  Yperen te
	PAGE_REFS: [295] 	CURR LEMMA: Pafporten
6 70 15 continue_stop 	     —— Uytvoeren.   295,
7 61 7 continue 	    —— 907 de Officicien van Halket om de
8 0 -53 start 	      —— Monteeringe van haar Regiment na Meenen
	PAGE_REFS: [297] 	CURR LEMMA: Pafporten
9 0 -53 start_stop 	 9     te mogen Voeren.  297.
10 52 -1 start 	    —— voor de Weduwe van Dam tot ayt-
11 98 41 continue 	      voer van Monteeringe voor bet Regiment van
	PAGE_REFS: [299] 	CURR LEMMA: Pafporten
12 91 34 continue_stop 	      ‘Caris na Meenen.   299.
13 9 -59 start 	 oo         voor Lemmens  tot den uytvoer van
14 101 22 continue 	      Monteeringe  voor  het  Regiment  van  den
15 101 21 continue 	      Kroonprins  van  Pruy(en  naar  Naamen.
16 63 -17 start 	    —— or  Buchwits  om  de  Monteeringe
17 102 21 continue 	      voor [ijn Regiment na Doornick. te moogen
	PAGE_REFS: [304] 	CURR

4 144 12 continue_stop 	        overgenomen.   198.
5 98 -35 start 	      ——  Mifive van den Marquis de Prié,
6 144 13 continue 	        affcheydt  neemende  van  haar  Hoogh Mo-
7 139 8 continue 	        gende ,  met  refcriptie  in civile termen be-
	PAGE_REFS: [199] 	CURR LEMMA: Pefters
8 143 16 continue_stop 	        antwoordt.   199.
9 104 -23 start 	       —— advis van den Raadt van Staateop -
HAS LEMMA: de Mifivens , raakende de nieuwe Admodia-
LEMMA: de Mifivens 
LEMMA: de Mifivens 
setting lemma: de Mifivens
10 131 -1 start 	        de Mifivens , raakende de nieuwe Admodia-
	PAGE_REFS: [204] 	CURR LEMMA: de Mifivens
11 142 10 continue_stop 	        teurs en refolutie.   204.
12 98 -29 start 	       gamwoordt  van  den  Grave Daun ,
13 144 17 continue 	        dat bet ver[chulde van de  oude en nieuwe
14 143 12 continue 	        admodiatie  aan  den  Staat [al werden be-
	PAGE_REFS: [209] 	CURR LEMMA: de Mifivens
15 145 18 continue_stop 	        taalt.   209.
16 98 -29 start 	 

30 121 10 continue 	     wegens rangh         den Advocaat Filaal
31 124 16 continue 	     en Procureur Generaal en den Griffier van
32 125 23 continue 	     den  Raadt  van  Brabanat  te
	PAGE_REFS: [459] 	CURR LEMMA: R
33 116 14 continue_stop 	     459.
34 69 -35 start 	  —  tegens  het  draagen van Meen met
	PAGE_REFS: [430] 	CURR LEMMA: R
35 106 3 continue_stop 	    Scherpe punten, te examineeren.  430.
36 68 -34 start 	  —— fubfidie van drie duyfent guldens voor
	PAGE_REFS: [490] 	CURR LEMMA: R
37 116 20 continue_stop 	     de exploitten.  490.
38 97 3 continue 	    ——    berichten op de Mifjive van Pefters
39 114 15 continue 	     om  nader elucidatie  op  de  rangh
	PAGE_REFS: [499] 	CURR LEMMA: R
40 114 15 continue_stop 	     den Fiftaal en Grifjier.   499.
41 65 -39 start 	  —— te advijeeren op bet verfoeck van Oo=
HAS LEMMA: Serbout ‚ Roftndaal en Nijpen om: Oftroy
setting lemma: Serbout
42 102 -2 start 	    Serbout ‚  Roftndaal en  Nijpen om: Oftroy
43 112 12 continue 	     

DEVIATING LINE: 228 [240, 105, 55, 105, 228, 105, 102] item op de geproponeerde van Pries-
avg_repeat_symbol_length: 153
0 64 -30 start 	    — om conferentie op de Requefte van den
1 114 17 continue 	       Paghter van de Brandewynen over ’s Her-
	PAGE_REFS: [210] 	CURR LEMMA: van Ofch
2 115 23 continue_stop 	       togenbofch.   210.
3 67 -27 start 	    —— item om middelen tot redres te bera-
4 112 16 continue 	       men wegens  vervallen Finantien in Plaan-
	PAGE_REFS: [210] 	CURR LEMMA: van Ofch
5 112 16 continue_stop 	       deren.  210.
6 63 -32 start 	    —— te advifeeren op het ver[oeck van Rez.
7 106 11 continue 	      genten van St. Michiels Geftel en Gemande
	PAGE_REFS: [211] 	CURR LEMMA: van Ofch
8 112 12 continue_stop 	       om redutlic van            211,
9 63 -31 start 	    —  rapport  van  de  Conferentie  wezens
10 111 17 continue 	       bet different tufchen de Raadt van
11 110 11 continue 	       en den Thefaurier Generaal Hop, te exami-
12 111 16 continue 	       

TypeError: list indices must be integers or slices, not NoneType

In [42]:
lemmas = [lemma for lemma in lemma_index if len(lemma) >= 5]


### Fuzzy Searching of Keywords in the Resolutions

Knowing which keywords should appear in the text, possibly with some spelling variation and OCR errors, we can use a fuzzy search algorithm to find candidate matches. 

Keywords that are similar to each other are registered as distractor terms, so matches are assigned as candidates to the nearest of sets of similar keywords. 

In [43]:
from republic.fuzzy.fuzzy_context_searcher import FuzzyContextSearcher, get_term_context

config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 3,
    "skip_size": 0,
}

fuzzy_lemma_searcher = FuzzyContextSearcher(config)

fuzzy_lemma_searcher.index_keywords(lemmas)

In [45]:
from republic.parser.republic_base_page_parser import merge_text_lines
from republic.model.republic_hocr_model import make_hocr_page

lemma_matches = defaultdict(list)

def add_context(match, page_text):
    context = get_term_context(page_text, match, context_size=40)
    match["match_term_in_context"] = context["match_term_in_context"]
    match["context_start_offset"] = context["start_offset"]
    match["context_end_offset"] = context["end_offset"]

for page_id in pages_info:
    page_doc = rep_es.retrieve_page_doc(es, page_id, year_config)
    print("\n\n", page_id)
    if page_doc["page_type"] == "index_page":
        print("skipping index page")
        continue
    for column_hocr in page_doc["columns"]:
        page_text = " ".join([line["line_text"] for line in column_hocr["lines"]])
        matches = fuzzy_lemma_searcher.find_candidates(page_text)
        for match in matches:
            lemma_matches[match["match_keyword"]] += [match]
            add_context(match, page_text)
            match["page_num"] = page_doc["page_num"]
            print(match["match_keyword"], "\t", match)
        #break




 year-1725-scan-1-odd


 year-1725-scan-4-odd


 year-1725-scan-5-odd
skipping index page


 year-1725-scan-6-even
skipping index page


 year-1725-scan-6-odd
skipping index page


 year-1725-scan-7-even
skipping index page


 year-1725-scan-7-odd
skipping index page


 year-1725-scan-8-even
skipping index page


 year-1725-scan-8-odd
skipping index page


 year-1725-scan-9-even
skipping index page


 year-1725-scan-9-odd
skipping index page


 year-1725-scan-10-even
skipping index page


 year-1725-scan-10-odd
skipping index page


 year-1725-scan-11-even
skipping index page


 year-1725-scan-11-odd
skipping index page


 year-1725-scan-12-even
skipping index page


 year-1725-scan-12-odd
skipping index page


 year-1725-scan-13-even
skipping index page


 year-1725-scan-13-odd
skipping index page


 year-1725-scan-14-even
skipping index page


 year-1725-scan-14-odd
skipping index page


 year-1725-scan-15-even
skipping index page


 year-1725-scan-15-odd
skipping index page


 yea

 year-1725-scan-46-even
Luycken 	 {'match_keyword': 'Luycken', 'match_term': 'Luycken', 'match_string': 'Laycken', 'match_offset': 668, 'char_match': 0.8571428571428571, 'ngram_match': 0.75, 'levenshtein_distance': 0.8571428571428572, 'match_term_in_context': 'eringe geleen de Requefte van Patroclus Laycken, Predikant by haar Hoogh Mogende Minifte', 'context_start_offset': 628, 'context_end_offset': 716, 'page_num': 92}
Opdorp 	 {'match_keyword': 'Opdorp', 'match_term': 'Opdorp', 'match_string': 'Opdorp', 'match_offset': 346, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'n. Ox een. Miflive van den Secre- taris Opdorp , gefchreven te Koppenha- ge den drie en', 'context_start_offset': 306, 'context_end_offset': 393, 'page_num': 92}
Denemarken 	 {'match_keyword': 'Denemarken', 'match_term': 'Denemarken', 'match_string': 'Deenemarcken', 'match_offset': 694, 'char_match': 1.0, 'ngram_match': 0.9090909090909091, 'levenshtein_distance': 0.833333

Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 249, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': ' Declaratie gefonden {al werden aan den Raadt van Staate ende des Gene- Reeckenkamer, ', 'context_start_offset': 209, 'context_end_offset': 295, 'page_num': 96}
Hollandt’ 	 {'match_keyword': 'Hollandt’', 'match_term': 'Hollandt’', 'match_string': 'Hollandt', 'match_offset': 1101, 'char_match': 0.8888888888888888, 'ngram_match': 0.8, 'levenshtein_distance': 0.8888888888888888, 'match_term_in_context': 'hippers-plaatfen van gemelde Stadt naar Hollandt en andere buytenlandtfche Stee- den, het', 'context_start_offset': 1061, 'context_end_offset': 1150, 'page_num': 96}
Commiflien 	 {'match_keyword': 'Commiflien', 'match_term': 'Commiflien', 'match_string': 'Commiflie', 'match_offset': 1681, 'char_match': 0.9, 'ngram_match': 0.8181818181818182, 'levenshtein_distance': 0.9, 'match_term_in_context':

 year-1725-scan-50-even
van Haaren 	 {'match_keyword': 'van Haaren', 'match_term': 'van Haaren', 'match_string': 'van haare', 'match_offset': 203, 'char_match': 0.8, 'ngram_match': 0.6363636363636364, 'levenshtein_distance': 0.8, 'match_term_in_context': 'rleeckeringe , dat haar Hoogh Mog. oock van haare zyde van in- tentie ende gerefolveert zy', 'context_start_offset': 163, 'context_end_offset': 253, 'page_num': 100}
Bruyninx 	 {'match_keyword': 'Bruyninx', 'match_term': 'Bruyninx', 'match_string': 'Bruyninx', 'match_offset': 848, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'rdt gefonden werden aan den Heere Hamel Bruyninx ‚ haar Hoogh Mogende extraordinaris Envo', 'context_start_offset': 808, 'context_end_offset': 897, 'page_num': 100}
Bruyninx 	 {'match_keyword': 'Bruyninx', 'match_term': 'Bruyninx', 'match_string': 'Bruyninx', 'match_offset': 1053, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_con

 year-1725-scan-52-even
Petitie ex Staat van Oorlogh 	 {'match_keyword': 'Petitie ex Staat van Oorlogh', 'match_term': 'Petitie ex Staat van Oorlogh', 'match_string': 'Petitie en Staat van oorlogh', 'match_offset': 225, 'char_match': 0.9285714285714286, 'ngram_match': 0.8620689655172413, 'levenshtein_distance': 0.9285714285714286, 'match_term_in_context': 'rftaan , dar de voorlchreve ge- neraale Petitie en Staat van oorlogh gefon- den fullen werden aan de Heeren S', 'context_start_offset': 185, 'context_end_offset': 294, 'page_num': 104}
Petitie ex Staat van Oorlogh 	 {'match_keyword': 'Petitie ex Staat van Oorlogh', 'match_term': 'Petitie ex Staat van Oorlogh', 'match_string': 'Petitie en Staat van oorlogh', 'match_offset': 494, 'char_match': 0.9285714285714286, 'ngram_match': 0.8620689655172413, 'levenshtein_distance': 0.9285714285714286, 'match_term_in_context': 'ten dien eynde de voorfchreve generaale Petitie en Staat van oorlogh peen over dagh en nacht fonder tydt verf', 'context_

...

Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 705, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': ' naam van den Raadt , ende met door den Raadt van Staate {elfs gefchreven, en da: op o', 'context_start_offset': 665, 'context_end_offset': 751, 'page_num': 375}
Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 794, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': ' tydr dat alle de abiente Leden van den Raadt befchreven waren, ende in vier of vyí da', 'context_start_offset': 754, 'context_end_offset': 840, 'page_num': 375}
Solutie 	 {'match_keyword': 'Solutie', 'match_term': 'Solutie', 'match_string': 'folutie', 'match_offset': 1017, 'char_match': 0.8571428571428571, 'ngram_match': 0.75, 'levenshtein_distance': 0.8571428571428572, 'match_term_in_context': 'e klagen, dat by haar Hoogh Mogende Re- fo

Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 892, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'ordre van hét Landt , hy het f{elve den Raadt moet aandienen , om daar op gelet en ger', 'context_start_offset': 852, 'context_end_offset': 938, 'page_num': 378}
Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 1130, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': ' acquifce- ren aan de Refolutie van den Raadt , ge- lijck in het tegenwoordige geval ,', 'context_start_offset': 1090, 'context_end_offset': 1176, 'page_num': 378}
Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 1288, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'oude wefen, of dar de Refolutie van den Raadt aan de ge van den Thefaurier gen

Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 2191, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'om ter faake daar by vermeldt, voor den Raadt van Sraate te procedeeren tegens den Ont', 'context_start_offset': 2151, 'context_end_offset': 2237, 'page_num': 381}
Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 2443, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'ren, dat voorfchreve Refolatien van den Raadt van Staate zynde genomen, niet tegen- 4 ', 'context_start_offset': 2403, 'context_end_offset': 2489, 'page_num': 381}


 year-1725-scan-191-even
Raadt 	 {'match_keyword': 'Raadt', 'match_term': 'Raadt', 'match_string': 'Raadt', 'match_offset': 395, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0, 'match_term_in_context': 'der te furneeren ter Secretarye van den Raadt van

Pefters 	 {'match_keyword': 'Pefters', 'match_term': 'Pefters', 'match_string': 'Pefers', 'match_offset': 295, 'char_match': 0.8571428571428571, 'ngram_match': 0.75, 'levenshtein_distance': 0.8571428571428572, 'match_term_in_context': 'llen. OQ een Miflive van den Refí- dent Pefers, gefchreven te Bruffel den defer Joopend', 'context_start_offset': 255, 'context_end_offset': 342, 'page_num': 386}
van Haaren 	 {'match_keyword': 'van Haaren', 'match_term': 'van Haaren', 'match_string': 'van haare', 'match_offset': 1189, 'char_match': 0.8, 'ngram_match': 0.6363636363636364, 'levenshtein_distance': 0.8, 'match_term_in_context': 'te verfoecken remifhie en dedommagement van haare over fwaare laften en geleden {chaade 5 ', 'context_start_offset': 1149, 'context_end_offset': 1239, 'page_num': 386}
Gedeputeerden 	 {'match_keyword': 'Gedeputeerden', 'match_term': 'Gedeputeerden', 'match_string': 'Gedeputeerden', 'match_offset': 1521, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 

KeyboardInterrupt: 

In [3]:
for lemma in sorted(lemma_matches):
    print("\n", lemma, "\tAantal kandidaten:", len(lemma_matches[lemma]), "\n")
    for match in lemma_matches[lemma]:
        print("\tKandidaat:", match["match_string"])
        print("\tPagina:", match["page_num"])
        print("\tContext:", match["match_term_in_context"][5:-5])
        print()


for lemma in lemma_index:
    print("\nTrefwoord:", lemma)
    #print(lemma_index[lemma])
    for entry in lemma_index[lemma]:
        pages = ", ".join([str(page_ref) for page_ref in entry["page_refs"]])
        description = entry["description"][:70]
        print("\tPagina:", pages, "\tBeschrijving:", description)

NameError: name 'lemma_matches' is not defined

In [ ]:
# scan 45 uneven is first resolution page
# page num: 91

from fuzzy_context_searcher import FuzzyContextSearcher

config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 3,
    "skip_size": 0,
}

fuzzy_searcher = FuzzyContextSearcher(config)

keywords = [
    "Admiraliteyt tot Amfterdam", 
    "Admiraliteyt in het Noorder Quartier", 
    "Admiraliteyt in Vrieslandt", 
    "Admiralteyt in Zeelandt",
    "Varckens"
]

distractor_terms = {
    "Admiraliteyt tot Amfterdam": {
        "Admiraliteyt in het Noorder Quartier", "Admiraliteyt in Vrieslandt", "Admiralteyt in Zeelandt"
    },
    "Admiraliteyt in het Noorder Quartier": {
        "Admiraliteyt tot Amfterdam", "Admiraliteyt in Vrieslandt", "Admiralteyt in Zeelandt"
    },
    "Admiraliteyt in Vrieslandt": {
        "Admiraliteyt tot Amfterdam", "Admiraliteyt in het Noorder Quartier", "Admiralteyt in Zeelandt"
    },
    "Admiralteyt in Zeelandt": {
        "Admiraliteyt tot Amfterdam", "Admiraliteyt in het Noorder Quartier", "Admiraliteyt in Vrieslandt"
    },
}
fuzzy_searcher.index_keywords(keywords)
fuzzy_searcher.index_distractor_terms(distractor_terms)

hocr_resolution_pages = []

